In [2]:
.include header

In [3]:
%%file header/Rosetta-Code-fft.h

#pragma once
#include <complex>
#include <valarray>

namespace Rosetta_Code {    
     
    template < class type >
        const type PI = static_cast<type>(atan(-1));//(3.141592653589793238460);
 
    template < class type > 
        using Complex = std :: complex<type> ;
        
    template < class type > 
        using  CArray = std :: valarray<Complex<type>> ;
        
    template < class type >
        void fft(CArray<type>& x);
    template < class type >
        void ifft(CArray<type>& x);
 
    // Cooley–Tukey FFT (in-place, divide-and-conquer)
    // Higher memory requirements and redundancy although more intuitive
    template < class type >
        void fft(CArray<type>& x){

            const size_t N = x.size();
            if (N <= 1) return;

            // divide
            CArray<type> even = x[std::slice(0, N/2, 2)];
            CArray<type>  odd = x[std::slice(1, N/2, 2)];

            // conquer
            fft(even);
            fft(odd);

            // combine
            for (size_t k = 0; k < N/2; ++k)
            {
                Complex<type> t = std::polar(1.0, -2 * PI<type> * k / N) * odd[k];
                x[k    ] = even[k] + t;
                x[k+N/2] = even[k] - t;
            }

            return ;

        };
    
    // inverse fft (in-place)
    template < class type >
        void ifft(CArray<type>& x){

            // conjugate the complex numbers
            x = x.apply(std::conj<type>);

            // forward fft
            fft( x );

            // conjugate the complex numbers again
            x = x.apply(std::conj);

            // scale the numbers
            x /= x.size();

        };
    
}

Overwriting header/Rosetta-Code-fft.h


In [7]:
%%file header/polynomial-multiplication.h

//#pragma once

//namespace eliastocco {

//    namespace polynomial {
    
//        template < class type , t_dim dimension , t_dim codimension >
//            polynomial<type,dimension,codimension>
//                /*polynomial<type,dimension,codimension> ::*/ operator * 
//                    ( const polynomial<type,dimension,codimension> left ,
//                         const polynomial<type,dimension,codimension> right ) 
                        {
                    
                        const unsigned int width = 15 ;
                        const unsigned int prec  = numDecimals ;
                                                
                        
                        // dovrei controllare che i polinomi siano definiti rispetto alla stessa base
                        if ( left._basis_function != right._basis_function ) {
                            assert ("polynomial :: operator * error; coeff array with different basis" );
                        }
                        auto basis = left._basis_function ;
                        // questo metodo di moltiplicare i polinomi funziona solo per polinomi definiti
                        // rispetto ad una base di monomi
                        // per adesso non aggiungo nessun controllo sul "tipo di base"
                        // perché dovrei implementare ancora tutto

                        // la condizione su dimension può essere indebolita
                        // ma per ora implementiamo la moltiplicazione fra polinomi
                        // solo nel caso 1D 
                        static_assert ( codimension == 1 and dimension == 1 );
                    
                        //ottengo gli array con i coeffcienti (reali)
                        auto left_coeff_real  = left.all_coeffs();
                        auto right_coeff_real = right.all_coeffs();  
                    
                        // eseguo più questo controllo perché l'algoritmo funziona anche se
                        // i coefficienti hanno lunghezza diversa
                        // l'importante è che allo stesso indice corrisponda lo stesso monomio
                    
                        //controllo che gli array abbiano la stessa dimensione
                        /*auto N = left_coeff_real.size();                    
                        if ( N != right_coeff_real.size() ){                            
                            assert ("polynomial :: operator * error; coeff array of different dimension" );                
                        }*/
                    
                        //modifico la dimensione degli array dei coefficienti 
                        // in modo tale che abbiano lunghezza pari ad una potenza di 2
                        unsigned int n = 1;
                        //while ( n < left_coeff_real.size() or n < right_coeff_real.size()){
                        while ( n < left_coeff_real.size() + right_coeff_real.size()){
                            n <<= 1;
                        }
                        
                        // non ne modifico la lunghezza,
                        // tanto i valori qui contenuti verranno copiati in due nuovi array
                    
                        //left_coeff_real.resize(n); // i nuovi valori saranno impostati a 0
                        //right_coeff_real.resize(n);// i nuovi valori saranno impostati a 0
                    
                        // efinisco gli array di coefficienti complessi
                        // che saranno gli input della funzione fft
                        typename Rosetta_Code :: CArray<type> left_coeff  (n);
                        typename Rosetta_Code :: CArray<type> right_coeff (n);

                        // copio i coefficienti e li trasformo in valori complessi
                        //left
                        for ( auto i = 0 ; i < left_coeff_real.size(); ++i ){
                            left_coeff[i] = Rosetta_Code :: Complex <type> ( left_coeff_real[i] , 0 );
                        }
                        //right
                        for ( auto i = 0 ; i < right_coeff_real.size(); ++i ){
                            right_coeff[i] = Rosetta_Code :: Complex <type> ( right_coeff_real[i] , 0 );
                        }
                        
                        if ( debug ) {   
                            std :: cout << "original arrays:" << std::setw(width)<< std :: endl;
                            for ( auto i = 0 ; i < n; ++i ){
                                std :: cout 
                                    <<left_coeff[i]  << std::setw(width)<< std::setprecision(prec)
                                    << " , " << std::setprecision(prec)
                                    <<right_coeff[i] << std::setw(width)<< std::setprecision(prec)
                                    << std :: endl ;                                     
                            }                        
                        }
                        
                        // eseguo le trasformate di Fourier dirette inplace  
                        Rosetta_Code :: fft ( left_coeff  );
                        Rosetta_Code :: fft ( right_coeff );         
                        
                        if ( debug ) {   
                            std :: cout << "fft arrays:" << std::setw(width)<< std :: endl;
                            for ( auto i = 0 ; i < n; ++i ){
                                std :: cout
                                    <<left_coeff[i]  << std::setw(width)<< std::setprecision(prec)
                                    << " , " << std::setprecision(prec)
                                    <<right_coeff[i] << std::setw(width)<< std::setprecision(prec)<< std :: endl ;                                     
                            }                        
                        }
                    
                        // moltiplico tra di loro i due vettori dei coefficienti nello spazio reciproco
                        // sfrutto il fatto che gli array di coefficienti siano dei std :: valarray
                        Rosetta_Code :: CArray<type> coeff = left_coeff * right_coeff;                    
                          
                        if ( debug ) {   
                            std :: cout << "fft multiplication:" << std::setw(width)<< std :: endl;
                            for ( auto i = 0 ; i < n; ++i ){
                                std :: cout
                                    <<coeff[i]  << std::setw(width)<< std::setprecision(prec)
                                    << std :: endl ;                                     
                            }                        
                        }
                    
                    
                        // eseguo la trasformata di Fourier inversa inplace                        
                        Rosetta_Code :: ifft( coeff );
                        
                        if ( debug ) {   
                            std :: cout << "new coeffs:" << std::setw(width) << std :: endl;
                            for ( auto i = 0 ; i < n; ++i ){
                                std :: cout 
                                    <<coeff[i]  << std::setw(width)<< std::setprecision(prec)
                                    << std :: endl ;                                     
                            }                        
                        }

                        // creo un polinomio e gli assegno i coefficienti calcolati
                        // solo se questi non sono nulli, 
                        // per ottimizzare spazio e rendere l'output più leggibile
                        polynomial<type,dimension,codimension> output ( basis ) ;
                        
                        
                        auto my_round = [=](const type& value ) {
                            const type multiplier = static_cast<type>(std::pow( 10.0, numDecimals ));
                            return static_cast<type>(std::ceil(value * multiplier) / multiplier );
                        };
                        
                        type c1 = type();
                        type c  = type();
                        for (auto i = 0; i < n; i++){
                            c1 = coeff [i] . real();
                            c = my_round(c1);
                            if ( c != type() ){                                
                                output.set(i,c,check);
                            }
                        }
                    
                        return output;
            
                } ; 
    
//    }
    
//}

Overwriting header/polynomial-multiplication.h


In [4]:
#include "Rosetta-Code-fft.h"
#include <iostream>

In [5]:
const Rosetta_Code::Complex<double> test[] = { 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0 };
Rosetta_Code::CArray<double>     data(test, 8);

In [6]:
// forward fft
Rosetta_Code::fft(data);

In [7]:
    std::cout << "fft" << std::endl;
    for (int i = 0; i < 8; ++i)
    {
        std::cout << data[i] << std::endl;
    }

fft
(4,0)
(3.73613,1.13334)
(0,0)
(0.35202,-0.658583)
(0,0)
(0.111625,-0.367977)
(0,0)
(-0.199779,-0.106784)


In [8]:
    // inverse fft
Rosetta_Code::ifft(data);

In [9]:

 

 
    std::cout << std::endl << "ifft" << std::endl;
    for (int i = 0; i < 8; ++i)
    {
        std::cout << data[i] << std::endl;
    }


ifft
(1,1.38778e-17)
(1,-4.16334e-17)
(1,5.55112e-17)
(1,-0)
(5.55112e-17,-1.38778e-17)
(0,4.16334e-17)
(5.55112e-17,-5.55112e-17)
(0,0)
